In [ ]:
from tensorflow.keras.layers import Conv2D, Flatten, Dense, MaxPooling2D, Dropout, Activation
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.models import Sequential
from tensorflow.keras.callbacks import ModelCheckpoint
import matplotlib.pyplot as plt

# Directory paths
train_dir = '../data/pneumonia/train/'
val_dir = '../data/pneumonia/val/'
test_dir = '../data/pneumonia/test/'

# Constants
batch_size = 16
epochs = 20
img_height, img_width = 150, 150
nb_train_samples = 5217
nb_validation_samples = 17

# Data generators
train_datagen = ImageDataGenerator(rescale=1./255, shear_range=0.2, zoom_range=0.2, horizontal_flip=True)
test_datagen = ImageDataGenerator(rescale=1./255)

train_data = train_datagen.flow_from_directory(train_dir, target_size=(img_height, img_width),
                                               batch_size=batch_size, class_mode='binary')

val_data = test_datagen.flow_from_directory(val_dir, target_size=(img_height, img_width),
                                           batch_size=batch_size, class_mode='binary')

test_data = test_datagen.flow_from_directory(test_dir, target_size=(img_height, img_width),
                                             batch_size=batch_size, class_mode='binary')

# Model definition
model = Sequential()
model.add(Conv2D(32, (3, 3), input_shape=(img_width, img_height, 3), activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Conv2D(32, (3, 3), activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Conv2D(64, (3, 3), activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Flatten())
model.add(Dense(64, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(1, activation='sigmoid'))

model.summary()

model.compile(loss='binary_crossentropy', metrics=['accuracy'], optimizer='rmsprop')

# Model checkpoint callback
mc = ModelCheckpoint('pnemonia_model_mc.h5', monitor="val_loss", verbose=1, save_best_only=False)

# Model training
history = model.fit(train_data, epochs=epochs, steps_per_epoch=nb_train_samples//batch_size,
                    validation_data=val_data, validation_steps=nb_validation_samples//batch_size,
                    callbacks=[mc])

# Save the model
model.save('../models/pneumonia_model.h5')

# Plot model performance
acc = history.history['accuracy']
val_acc = history.history['val_accuracy']
loss = history.history['loss']
val_loss = history.history['val_loss']
epochs_range = range(1, len(history.epoch) + 1)

plt.figure(figsize=(15, 5))

plt.subplot(1, 2, 1)
plt.plot(epochs_range, acc, label='Train Set')
plt.plot(epochs_range, val_acc, label='Val Set')
plt.legend(loc="best")
plt.xlabel('Epochs')
plt.ylabel('Accuracy')
plt.title('Model Accuracy')

plt.subplot(1, 2, 2)
plt.plot(epochs_range, loss, label='Train Set')
plt.plot(epochs_range, val_loss, label='Val Set')
plt.legend(loc="best")
plt.xlabel('Epochs')
plt.ylabel('Loss')
plt.title('Model Loss')

plt.tight_layout()
plt.show()

# Evaluate the model on the test set
scores = model.evaluate_generator(test_data)
print("\n%s: %.2f%%" % (model.metrics_names[1], scores[1] * 100))
